In [1]:
import numpy as np 
import pandas as pd

from PIL import Image

import torch
import torch.nn as nn
import torch.utils.data as D
import torch.nn.functional as F

import albumentations
import torchvision
from torchvision import transforms

import tqdm

from src.losses import lovasz as L

from src.utils.metrics import accuracy
from src.utils.data_loaders import ImagesDS
from matplotlib import pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [2]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [3]:
path_data = '/home/vladislav/Data'
device = 'cuda'
batch_size = 128

In [4]:
transforms_train = albumentations.Compose([
    albumentations.RandomCrop(height=334, width=334, p=1),
    albumentations.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=0, p=0.3),
    albumentations.RandomRotate90(p=0.3),
    albumentations.Flip(p=0.3),
    albumentations.Normalize(
     mean=[0.01639522, 0.06704897, 0.0366343 , 0.03555733, 0.02223296, 0.03011626],
     std=[0.01954379, 0.05231356, 0.01980171, 0.02657354, 0.01802711, 0.01548923])
])

transforms_test = albumentations.Compose([
    albumentations.RandomCrop(height=334, width=334, p=1),
    albumentations.Normalize(
     mean=[0.01639522, 0.06704897, 0.0366343 , 0.03555733, 0.02223296, 0.03011626],
     std=[0.01954379, 0.05231356, 0.01980171, 0.02657354, 0.01802711, 0.01548923])
])

In [5]:
ds = ImagesDS(pd.read_csv(path_data+'/train.csv'), path_data, transforms_train, mode='train')
ds_test = ImagesDS(pd.read_csv(path_data+'/test.csv'), path_data, transforms_test, mode='test')

loader = D.DataLoader(ds, batch_size=batch_size, shuffle=True, num_workers=8)
tloader = D.DataLoader(ds_test, batch_size=batch_size, shuffle=False, num_workers=8)

In [6]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Parameter
import math


class ArcMarginProduct(nn.Module):
    r"""Implement of large margin arc distance: :
        Args:
            in_features: size of each input sample
            out_features: size of each output sample
            s: norm of input feature
            m: margin
            cos(theta + m)
        """
    def __init__(self, in_features, out_features, s=32.0, m=0.50, easy_margin=False):
        super(ArcMarginProduct, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.s = s
        self.m = m
        self.weight = Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

        self.easy_margin = easy_margin
        self.cos_m = math.cos(m)
        self.sin_m = math.sin(m)
        self.th = math.cos(math.pi - m)
        self.mm = math.sin(math.pi - m) * m

    def forward(self, input, label):
        # --------------------------- cos(theta) & phi(theta) ---------------------------
        cosine = F.linear(F.normalize(input), F.normalize(self.weight))
        sine = torch.sqrt((1.0 - torch.pow(cosine, 2)).clamp(0, 1))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = torch.where(cosine > 0, phi, cosine)
        else:
            phi = torch.where(cosine > self.th, phi, cosine - self.mm)
        # --------------------------- convert label to one-hot ---------------------------
        # one_hot = torch.zeros(cosine.size(), requires_grad=True, device='cuda')
        one_hot = torch.zeros(cosine.size(), device='cuda')
        one_hot.scatter_(1, label.view(-1, 1).long(), 1)
        # -------------torch.where(out_i = {x_i if condition_i else y_i) -------------
        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)  # you can use torch.where if your torch.__version__ is 0.4
        output *= self.s
        return output

In [7]:
model = torchvision.models.resnet34(True)
tail = nn.Conv2d(6, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
tail.weight.data[:,:3] = model.conv1.weight.data
tail.weight.data[:,3:] = model.conv1.weight.data
model.conv1 = tail
model.fc = nn.Sequential(
                          nn.Dropout(p=0.5),
                          nn.Linear(in_features=512, out_features=512, bias=True),
                          nn.BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                         )
model.to(device);

In [8]:
# train_df = pd.read_csv(path_data+'/train.csv')
# train_df_HEPG2 = train_df[train_df['experiment'].isin(['HEPG2-01', 'HEPG2-02', 'HEPG2-03', 'HEPG2-04', 'HEPG2-05',
#                                                        'HEPG2-06', 'HEPG2-07'])]

# ds_train = ImagesDS(train_df_HEPG2, path_data, transforms_train, mode='train')
# loader = D.DataLoader(ds_train, batch_size=batch_size, shuffle=True, num_workers=4)

In [ ]:
criterion = nn.CrossEntropyLoss()
metric_fc = ArcMarginProduct(512, 1108)
optimizer = torch.optim.Adam([{'params': model.parameters()}, {'params': metric_fc.parameters()}], 30e-5)
metric_fc.to(device)

epochs = 50
tlen = len(loader)

for epoch in range(epochs):
    model.train()
    tloss = 0
    acc = np.zeros(1)
    for data in tqdm.tqdm_notebook(loader):
        data_input, label = data
        data_input = data_input.to(device)
        label = label.to(device).long()
        feature = model(data_input)
        output = metric_fc(feature, label)
        loss = criterion(output, label)
        tloss += loss.item() 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        output = output.data.cpu().numpy()
        output = np.argmax(output, axis=1)
        label = label.data.cpu().numpy()
        acc += np.mean((output == label).astype(int))

    if epoch > 15:
        optimizer.param_groups[0]['lr'] = 15e-5
    if epoch > 20:
        optimizer.param_groups[0]['lr'] = 7.5e-5
    if epoch > 25:
        optimizer.param_groups[0]['lr'] = 3e-5
    if epoch > 35:
        optimizer.param_groups[0]['lr'] = 1e-5
        
    print('Epoch {} -> Train Loss: {:.4f}, ACC: {:.2f}%'.format(epoch+1, tloss/tlen, acc[0]/tlen))


Epoch 1 -> Train Loss: 22.2691, ACC: 0.00%



Epoch 2 -> Train Loss: 18.8818, ACC: 0.00%



Epoch 3 -> Train Loss: 13.9238, ACC: 0.00%


In [53]:
# torch.save(model.state_dict(), '/home/vladislav/PycharmProjects/untitled/data/models/densnet_v0/densnet201_arcface.pth')

---

In [14]:
def cosin_metric(x1, x2):
    return np.dot(x1, x2) / (np.linalg.norm(x1) * np.linalg.norm(x2))

In [43]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedKFold, KFold, train_test_split

In [55]:
X = []
y = []

model.eval()
with torch.no_grad():
    for data in tqdm.tqdm_notebook(loader):
        data_input, label = data
        data_input = data_input.to(device)
        feature = model(data_input)
        X.extend([i.cpu().numpy() for i in feature])
        y.extend([int(i) for i in data[1]])

In [56]:
X = np.array(X)
y = np.array(y)

In [58]:
n_folds = 4
folds = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=0)
fold = folds.split(X, y)

for fold_, (train_index, val_index) in enumerate(fold):
    clf = KNeighborsClassifier(n_neighbors=1, metric='cosine')
    clf.fit(X[train_index], y[train_index])
    print(sum(clf.predict(X[val_index]) == y[val_index])/len(val_index))

0.014453477868112014
0.015808491418247517
0.01625526791089705
0.015352197471402769


In [76]:
cosin_metric(X[0], X[1])

0.92886055

In [61]:
y[0]

841

In [74]:
y[0]

841

In [77]:
y[]

array([ 841,  149,  170, ..., 1101,  916,   17])